## Data Annotation Homework EDA
В данном документе описываются структура, способ сбора и аннотации данных по практическому заданию #2 курса DC-AI.

In [185]:
import numpy as np
import pandas as pd
import matplotlib as mtl
import matplotlib.pyplot as plt
from sklearn.metrics import cohen_kappa_score
import os
import json

В рамках выполнения задачи собраны следующие датасеты:
- `data/websites.json` - набор распаршенных сайтов, по которым собраны скриншоты,
- `data/data_markup_1/*` - набор размечанных данных Разметчиком #1 по скриншотам датасета websites,
- `data/data_markup_2/*` - набор размечанных данных Разметчиком #2 по скриншотам датасета websites,
- `data/data_markup_review.csv` - данные о корректности разметки по каждому элементу датсета websites.

### Датасет websites

Датасет собран с сайта `similarweb.com` путем парсинга рейтинга самых популярных сайтов. По каждому сайту в датасете веб-скраппер сделал скиншот, сохраненный в папке `data/screenshots/`. В датасет не включены сайты, доступ к которому веб-скраперу не удалось получить или попытка загрузить сайт закончилась ошибкой. 

In [186]:
websites = pd.read_json('data/websites.json');
print(websites.info())
websites.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   position  109 non-null    int64 
 1   name      109 non-null    object
 2   url       109 non-null    object
 3   category  109 non-null    object
 4   path      109 non-null    object
dtypes: int64(1), object(4)
memory usage: 4.4+ KB
None


,position,name,url,category,path
0,2,zillow.com,https://zillow.com/,business-and-consumer-services,screenshots/8c9df540_https___zillow.com_.png
1,3,usps.com,https://usps.com/,business-and-consumer-services,screenshots/4e4f3456_https___usps.com_.png
2,6,ups.com,https://ups.com/,business-and-consumer-services,screenshots/232df189_https___ups.com_.png
3,9,realtor.com,https://realtor.com/,business-and-consumer-services,screenshots/3d7521e8_https___realtor.com_.png
4,11,shopify.com,https://shopify.com/,business-and-consumer-services,screenshots/6a17b4bb_https___shopify.com_.png


В датасете `websites` представлена следующая структура данных:
- `position` - положение сайта в рейтинге топ-сайтов согласно данным similarweb.com,
- `name` - хостнейм сайта,
- `url` - ссылка на сайт,
- `category` - название категории, к которой отнесен сайт согласно своейсфере деятельности по данным similarweb.com,
- `path` - относительный путь до скриншота сайта от места хранения датасета, является уникальным идентификатором. 

### Датасеты разметки
Датасеты разметки от двух разметчиков представлены в виде json файлов в папках `data/data_markup_1` и `data/data_markup_2/`. 

In [187]:
ls 'data/data_markup_1/' | wc -l && ls 'data/data_markup_2/' | wc -l && echo " " && ls 'data/data_markup_1/' | head -n 5

107
109
 
0261b53e_https___kompas.com_.json
034c98a7_https___roblox.com_.json
03a9e3fa_https___otzovik.com_.json
060df3d1_https___nexusmods.com_.json
0a44f209_https___hoyoverse.com_.json


В датасетах `data_markup_1` и `data_markup_2` содержится разметка скриншотов в формате json. Название файла является идентификатором, по которому можно найти оригинальный скриншот и информацию о сайте в датасете websites.

Выше приведен пример названий файлов. 

Так же стоит отметить, что в датасете Разметчика #1 отсутствует разметка двух сайтов, т.к. данные содержат битые скриншоты с плохим качеством. 

In [188]:
dir_path = 'data/data_markup_1/'
files = os.listdir(dir_path)
with open(os.path.join(dir_path, files[0]), 'r') as file:
    data = json.dumps(json.load(file), indent=4)
    print(data)

{
    "version": "5.3.1",
    "flags": {},
    "shapes": [
        {
            "label": "button",
            "points": [
                [
                    1303.9999999999998,
                    2097.916666666667
                ],
                [
                    4031.083333333333,
                    2254.166666666667
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "rectangle",
            "flags": {}
        },
        {
            "label": "button",
            "points": [
                [
                    1301.9166666666665,
                    2266.666666666667
                ],
                [
                    4031.083333333333,
                    2435.416666666667
                ]
            ],
            "group_id": null,
            "description": "",
            "shape_type": "rectangle",
            "flags": {}
        }
    ],
    "imagePath": "f6cfd89e_https___gazzetta.it_.p

В файле разметки содержатся следующище необходимые для обучения модели данные:
- `version` - версия программного обеспечения LabelMe,
- `shapes` - массив аннотаций с названием лейблов.

Если объединить скриншоты в папке `./data/screenshots/` с разметкой в папке `./data/data_markup_1/` или `./data/data_markup_2/`, то с помощью LabelMe можно просмотреть разметку. Для примера в папку `./data/labelme_preview_example` помещен скриншот и его разметка. Пример интерфейса при открытии папки в LabelMe приведен ниже. 

![button-annotation-example.png](./assets/button-annotation-example.png)

### Анализ ответов разметчиков

In [189]:
answers = pd.read_csv('data/data_markup_review.csv')
print(answers.info())
answers.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         109 non-null    object 
 1   pass_total   107 non-null    float64
 2   pass_1       108 non-null    float64
 3   fail_1       111 non-null    float64
 4   pass_2       108 non-null    float64
 5   fail_2       111 non-null    float64
 6   broken_data  110 non-null    float64
 7   Unnamed: 7   0 non-null      float64
 8   0            109 non-null    float64
dtypes: float64(8), object(1)
memory usage: 7.9+ KB
None


,name,pass_total,pass_1,fail_1,pass_2,fail_2,broken_data,Unnamed: 7,0
0,0261b53e_https___kompas.com_.png,21.0,20.0,1.0,12.0,9.0,0.0,NaN,1.0
1,034c98a7_https___roblox.com_.png,2.0,2.0,0.0,2.0,0.0,0.0,NaN,1.0
2,03a9e3fa_https___otzovik.com_.png,2.0,2.0,0.0,2.0,0.0,0.0,NaN,1.0
3,060df3d1_https___nexusmods.com_.png,30.0,30.0,0.0,9.0,21.0,0.0,NaN,1.0
4,0a44f209_https___hoyoverse.com_.png,13.0,12.0,1.0,13.0,0.0,0.0,NaN,1.0


В датасете содержаться следующие поля:
- `name` - название скриншота, по которому осуществлялась разметка,
- `pass_1` - количество корректных аннотаций разметчиком 1,
- `fail_1` - количество некорректный и пропущенных аннотаций разметчиком 1,
- `pass_2` - количество корректных аннотаций разметчиком 2,
- `fail_2` - количество некорректный и пропущенных аннотаций разметчиком 2,
- `pass_total` - общее количество объектов на скриншоте, которые требуется аннотировать, 
- `broken_data` - является ли скриншот битым.

Далее требуется провести очистку данных для дальнейшего анализа.

In [190]:
answers = answers[answers['broken_data'] == 0]
answers_cropped = answers.drop(['broken_data', '0', 'Unnamed: 7'], axis=1)
answers_cropped['result_1'] = answers_cropped['fail_1'] == 0
answers_cropped['result_2'] = answers_cropped['fail_2'] == 0

print(cohen_kappa_score(answers_cropped['result_1'], answers_cropped['result_2']))

0.03381125615236469


Если при расчете согласованности считать за корретный результат разметки точную разметку всех элементов на странице, тогда рейтинг согласованности разметчиков будет крайне низким и равным ~0.0338. 

Однако при такого рода рассчетах не учитывается единичные случае согласованности при разметке объектов внутри одной страницы. Т.е. например на странице находится 20 объектов, один из разметчиков разметил 20 корректно, второй 19 корректно и 1 лишнию, тогда рейтинг согласованности будет крайне высоким, однако в общей статистике первый разметчик будем иметь корректный результат разметки, а второй разметчик - некорректный.

Для корректного расчета коэффициента Каппа Коэна по каждому объекту требуется проведение дополнительного анализа данных, выходящего за скоуп задачи.  